# Chapter 24: Class Metaprogramming

Class metaprogramming is the art of creating or customizing classes at runtime.

### Classes as Objects

Like most program entities in Python, classes are also objects. Every class has a number of attributes defined in the Python Data Model.

In [2]:
# Define a simple class
class Dog:
    species = "Canis familiaris"

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def speak(self, sound):
        return f"{self.name} says {sound}"

In [5]:
# Instantiating a Dog object
buddy = Dog("Buddy", 9)
print(buddy.speak("Woof"))

# Classes are objects too. Let's prove it.
print(isinstance(Dog, object))  # True

# Define a function that accepts a class as an argument and creates an instance
def create_pet(pet_class, name, age):
    return pet_class(name, age)

# Use the create_pet function to create an instance of Dog
my_pet = create_pet(Dog, "Molly", 3)
print(my_pet.speak("Woof Woof"))

# Adding an attribute to the Dog class object dynamically
Dog.breed = "Labrador"

# Now all instances of Dog, as well as the class itself, have the 'breed' attribute
print(Dog.breed)  # Labrador
print(buddy.breed)  # Labrador

Buddy says Woof
True
Molly says Woof Woof
Labrador
Labrador


### A Class Factory Function

The standard library has a class factory function that appears several times in this
book: collections.namedtuple.

In [8]:
def class_factory(class_name, say_what):
    # Define a new class dynamically
    class DynamicClass:
        def __init__(self, name):
            self.name = name

        def speak(self):
            return f"{self.name} says {say_what}!"

    # Set the __name__ attribute of the class to the given class_name
    DynamicClass.__name__ = class_name

    # Return the new class
    return DynamicClass



In [9]:
# Use the factory function to create a new class
GreeterClass = class_factory("Greeter", "Hello")

# Create an instance of the dynamically created class
greeter = GreeterClass("Alice")

# Use the instance
print(greeter.speak()) 

Alice says Hello!


### Introducing __init_subclass__

In [1]:
from checkedlib import Plugin

class PluginA(Plugin):
    pass

class PluginB(Plugin):
    pass


available_plugins = Plugin.get_plugins()
for plugin in available_plugins:
    print(plugin.__name__)
    

PluginA
PluginB
